# EDA — Precios las de Rentas en Guadalajara (DataScience)
Exploración inicial del dataset `gdl_rental_prices.csv` para preparar la app en Streamlit.


In [1]:
import pandas as pd
import plotly.express as px

pd.set_option("display.max_columns", 100)


In [2]:
CSV_PATH = "../gdl_rental_prices.csv"  # ajusta si moviste el archivo
# Si tu CSV tiene acentos/ñ y falla, prueba encoding="utf-8" o "latin-1"
df = pd.read_csv(CSV_PATH, encoding="utf-8")
df.head()


,no,location,price,bedrooms,bathrooms,garage,area
0,0,Zapopan,22300,3,4,2,180
1,1,Real Vallarta,12500,2,2,2,100
2,2,Puertas del Tule,25000,4,4,2,187
3,3,Zapopan,58000,3,4,4,700
4,4,Heroes Nacionales,9000,3,2,2,110


In [3]:
display(df.info())
display(df.describe(include="all"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   no         418 non-null    int64 
 1   location   418 non-null    object
 2   price      418 non-null    int64 
 3   bedrooms   418 non-null    int64 
 4   bathrooms  418 non-null    int64 
 5   garage     418 non-null    int64 
 6   area       418 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 23.0+ KB


None

,no,location,price,bedrooms,bathrooms,garage,area
count,418.000000,418,418.000000,418.000000,418.000000,418.000000,418.000000
unique,NaN,140,NaN,NaN,NaN,NaN,NaN
top,NaN,Zapopan,NaN,NaN,NaN,NaN,NaN
freq,NaN,48,NaN,NaN,NaN,NaN,NaN
mean,220.093301,NaN,24274.770335,2.877990,2.521531,2.153110,190.868421
std,126.072563,NaN,20219.998823,0.814699,0.890001,0.801977,158.816046
min,0.000000,NaN,3500.000000,0.000000,1.000000,1.000000,1.000000
25%,115.250000,NaN,12500.000000,2.000000,2.000000,2.000000,95.000000
50%,220.500000,NaN,19000.000000,3.000000,2.000000,2.000000,133.000000
75%,327.750000,NaN,27500.000000,3.000000,3.000000,2.000000,200.000000


In [4]:
num_cols = df.select_dtypes(include="number").columns.tolist()
cat_cols = df.select_dtypes(exclude="number").columns.tolist()
print("Numéricas:", num_cols)
print("Categóricas:", cat_cols)


Numéricas: ['no', 'price', 'bedrooms', 'bathrooms', 'garage', 'area']
Categóricas: ['location']


**Nota:** si `price`/`renta`/`m2` vienen como texto, conviértelos a numérico.


In [5]:
# Intenta convertir columnas típicas a numérico si existen
candidatas = ["price","rent","precio","renta","surface","m2","area","rooms","bedrooms","bathrooms"]
for c in candidatas:
    if c in df.columns:
        df[c] = (
            df[c].astype(str)
                 .str.replace(r"[^\d\.\-]", "", regex=True)  # quita $, comas, espacios
                 .replace({"": pd.NA})
                 .astype(float)
        )

num_cols = df.select_dtypes(include="number").columns.tolist()
num_cols


['no', 'price', 'bedrooms', 'bathrooms', 'garage', 'area']

In [8]:
fig = px.histogram(df, x="price", nbins=30, title="Histograma de rentas")
fig.show()

In [9]:
fig = px.scatter(df, x="area", y="price", title="Precio vs. Área")
fig.show()

## Hallazgos iniciales

- **Columnas numéricas detectadas:** `price`, `bedrooms`, `bathrooms`, `garage`, `area`.  
  La columna `no` solo es un índice, no aporta valor analítico.

- **Columna categórica:** `location` (140 valores distintos, la más frecuente es *Zapopan*).

- **Distribución de precios:** 
  - Renta mínima registrada ≈ $3,500  
  - Renta máxima registrada ≈ $150,000  
  - La mayoría de propiedades se concentran entre $10,000 y $30,000.

- **Superficie (area):**
  - Media ≈ 191 m², con propiedades pequeñas de 1 m² hasta grandes de 897 m².  
  - Valores extremos (outliers) que podrían influir en la dispersión.

- **Relaciones observadas:**
  - Existe correlación positiva entre **área** y **precio**: a mayor tamaño, suelen ser más caras.  
  - Algunas propiedades muy caras parecen ser outliers, no siguen la tendencia general.

- **Próximos pasos:**
  - Confirmar semántica de cada columna (ej. `garage` = número de cajones?).  
  - Usar `price` para el histograma y `area` vs `price` para la dispersión en la app de Streamlit.  
  - Explorar posibles comparaciones entre colonias (`location`) y precios.

